In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from tqdm import tqdm_notebook

In [ ]:
#   Ignore warning
import warnings
warnings.filterwarnings("ignore")

In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
#   Preprocessing data
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
#   Normalize (0 : 1)
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

In [4]:
#   Define dimension of noise input from Generator
z_dim = 100

#   Define optimizer
opt = Adam(learning_rate = 0.001)

In [5]:
G = Sequential()
G.add(Dense(256, input_shape = (z_dim, ), activation = LeakyReLU(), kernel_regularizer = l2(0.0001)))
G.add(BatchNormalization())
G.add(Dense(512, activation = LeakyReLU(), kernel_regularizer = l2(0.0001)))
G.add(BatchNormalization())
G.add(Dense(1024, activation = LeakyReLU(), kernel_regularizer = l2(0.0001)))
G.add(BatchNormalization())
G.add(Dense(784, activation = 'sigmoid'))
G.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

D = Sequential()
D.add(Dense(1024, input_shape = (784, ), activation = LeakyReLU(), kernel_regularizer = l2(0.0001)))
D.add(BatchNormalization())
D.add(Dense(512, activation = LeakyReLU(), kernel_regularizer = l2(0.0001)))
D.add(BatchNormalization())
D.add(Dense(256, activation = LeakyReLU(), kernel_regularizer = l2(0.0001)))
D.add(BatchNormalization())
D.add(Dense(1, activation = 'sigmoid'))
D.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [6]:
input = Input(shape = (z_dim,))
gen = G(input)
dis = D(gen)
gan = Model(inputs = input, outputs = dis)
gan.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 784)               1493520   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 1467393   
                                                                 
Total params: 2,960,913
Trainable params: 2,953,745
Non-trainable params: 7,168
_________________________________________________________________


In [15]:
def plot_generator(ex_dim, z_dim):
    z_noise = np.random.randn(ex_dim, z_dim)  
    generated_imgs = G.predict(z_noise)
    # generated_imgs = generated_imgs.reshape(ex_dim, 28, 28)
    # fig = plt.figure(figsize = (12, 2))
    # for index, image in enumerate(generated_imgs):
    #     ax = fig.add_subplot(10, 1, index+1)
    #     ax.imshow(generated_imgs)
    #     ax.axis('off')
    # plt.show()
    print(generated_imgs.shape)

In [11]:
#   Save loss and accuracy of Generator & Discriminator while training
loss = {'G': [], 'D': []}
def train(epochs, batch_size, plot_freq, z_dim):
    iters = X_train.shape[0] // batch_size
    for epoch in tqdm_notebook(range(1, epochs + 1)):
        if epoch == 1 or epoch % plot_freq == 0:
            print('-'*25, f'Epoch {epoch}', '-'*25)
        for _ in range(iters):
            #   Random z_noise from any random distributions (Standard Normal Distribution for this scenario)
            z_noise = np.random.randn(batch_size, z_dim)  
            #   Take random batch_size image from mnist dataset
            mnist_imgs = X_train[np.random.choice(X_train.shape[0], batch_size), :]

            #   Use Generator to generate new image from noise
            generated_imgs = G.predict(z_noise)
            X_discr = np.concatenate((mnist_imgs, generated_imgs))
            #   Create labels
            Y_discr = np.zeros((2*batch_size, 1))    #   Label 0 for generated images
            Y_discr[:batch_size] = 1

            #   Train Discriminator
            D.trainable = True
            D_loss = D.train_on_batch(X_discr, Y_discr)

            #   Train Generator
            D.trainable = False
            Y_gener = np.ones((batch_size))
            G_loss = gan.train_on_batch(z_noise, Y_gener)

            loss['D'].append(D_loss)
            loss['G'].append(G_loss)
        if epoch == 1 or epoch % plot_freq == 0:
            plot_generator(10, z_dim)

In [ ]:
train(epochs = 200, batch_size = 64, plot_freq = 20, z_dim = 100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/200 [00:00<?, ?it/s]

------------------------- Epoch 1 -------------------------
(10, 784)
